In [1]:
from pyspark import SparkContext


In [2]:
sc = SparkContext.getOrCreate()

25/11/24 17:04:30 WARN Utils: Your hostname, spark resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/11/24 17:04:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 17:04:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/24 17:04:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
file_path = "/home/spark/Downloads/archive/Iris.csv"

In [4]:
rdd = sc.textFile(file_path)

In [5]:
header = rdd.first()
rdd = rdd.filter(lambda line: line != header)

In [6]:
species_rdd = rdd.map(lambda line: line.split(",")[-1])

In [7]:
counts = species_rdd.map(lambda s: (s, 1)).reduceByKey(lambda a, b: a + b)

In [8]:
print("Türlere Göre Çiçek Sayısı:")
for species, count in counts.collect():
    print(f"{species}: {count}")

Türlere Göre Çiçek Sayısı:


Iris-setosa: 50
Iris-versicolor: 50
Iris-virginica: 50


In [9]:
def parse_line(line):
    parts = line.split(",")
    species = parts[-1]  # Tür
    sepal_length = float(parts[0]) 
    return (species, sepal_length)

In [10]:
species_lengths = rdd.map(parse_line)

In [11]:
species_stats = species_lengths.mapValues(lambda x: (x, 1)) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

In [12]:
averages = species_stats.mapValues(lambda x: x[0] / x[1])

In [13]:
print("\nHer Türün Ortalama Sepal Uzunluğu:")
for species, avg in averages.collect():
    print(f"{species}: {avg:.2f} cm")


Her Türün Ortalama Sepal Uzunluğu:


Iris-setosa: 25.50 cm
Iris-versicolor: 75.50 cm
Iris-virginica: 125.50 cm


In [14]:
max_lengths = species_lengths.reduceByKey(lambda a, b: max(a, b))
print("\nHer Türün En Uzun Sepal'i:")
for species, max_len in max_lengths.collect():
    print(f"{species}: {max_len} cm")


Her Türün En Uzun Sepal'i:


Iris-setosa: 50.0 cm
Iris-versicolor: 100.0 cm
Iris-virginica: 150.0 cm


In [15]:
long_sepal_rdd = rdd.filter(lambda line: float(line.split(",")[0]) > 6.0)
species_long = long_sepal_rdd.map(lambda line: line.split(",")[-1])
counts_long = species_long.map(lambda s: (s, 1)).reduceByKey(lambda a, b: a + b)

print("\nSepal Uzunluğu 6 cm'den Uzun Olanlar:")
for species, count in counts_long.collect():
    print(f"{species}: {count} adet")


Sepal Uzunluğu 6 cm'den Uzun Olanlar:


Iris-setosa: 44 adet
Iris-versicolor: 50 adet
Iris-virginica: 50 adet


In [16]:
sorted_counts = counts.sortBy(lambda x: x[1], ascending=False)

print("\nÇoktan Aza Sıralı Türler:")
for species, count in sorted_counts.collect():
    print(f"{species}: {count}")


Çoktan Aza Sıralı Türler:
Iris-setosa: 50
Iris-versicolor: 50
Iris-virginica: 50
